In [7]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [48]:
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath)as f:
    shakespeare_text = f.read()

In [49]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level = True)
tokenizer.fit_on_texts([shakespeare_text])

In [50]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [51]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

train_size = dataset_size * 90//100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [52]:
n_steps = 100
window_length = n_steps +1 
dataset = dataset.window(window_length, shift = 1, drop_remainder = True)

In [53]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [54]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [55]:
dataset = dataset.map(
lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth = max_id), Y_batch))

In [57]:
dataset = dataset.prefetch(1)

In [58]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

In [59]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences = True, input_shape = [None, max_id], dropout = 0.2, recurrent_dropout = 0.2), 
    keras.layers.GRU(128, return_sequences = True, input_shape = [None, max_id], dropout = 0.2, recurrent_dropout = 0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation = "softmax"))
])

In [61]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam")

In [64]:
#this is very slow to run, approx 2hrs per epoch using the colab TPU. 

#history = model.fit(dataset, steps_per_epoch=train_size // batch_size,epochs=10)